In [1]:
import pandas as pd

In [2]:
from bs4 import BeautifulSoup
import urllib.request


In [3]:
LIKELY_FORMATIONS="likely-formations.html"
ME="me"
THRESHOLD=35
bought=pd.read_csv("availability-of-players.csv")
import re
def download_html():
	try:
		html=urllib.request.urlopen ("https://www.fantacalcio.it/probabili-formazioni-serie-a").read()
		print("Server Online")
		with open(LIKELY_FORMATIONS,"w") as formations_file:
		
			formations_file.write(str(html))
	except urllib.error.HTTPError as e:
		print("Server Offline")
		#do stuff here
	except urllib.error.URLError as e:
		print("Server Offline")
        
def retrieve_players_html():
	with open(LIKELY_FORMATIONS,"r") as formations_file:
		return formations_file.read()

def probabilityOFEnteringLate(player_name,parsed_html):
    for a in parsed_html.find_all('a'):
        if player_name in a.contents:
            try:
                parent=a.parent.parent
                
                parent=parent.contents[0]
                indexOnString=str(parent).find("I.S.")
                cont=str(parent)[indexOnString+4:indexOnString+8]
                c2=re.findall(r"[0-9]+",cont)
                return float(c2[0])
                print(c2)
                content=parent.contents[1]

                content=content.strip("I.S. ")
                content=content.strip("%")
                content=content.strip(" ")
                percentage=content
                return float(percentage)
            except AttributeError:
                return 0
def probability_of_being_in_first_team(player_name,parsed_html):
    for a in parsed_html.find_all('a'):
        if player_name in a.contents:
            try:
                parent=a.parent.parent
                parent=parent.contents[1]
                parent=parent.contents[0]
                parent=parent.contents[0] #90%

                percentage=parent.contents[0]
                return (float(percentage.strip("%")))
            except AttributeError:
                return probabilityOFEnteringLate(player_name,parsed_html)
    return 0


def first_team(bought):
    html=str(retrieve_players_html())
    parsed_html = BeautifulSoup(html, 'html.parser')
    all={}
    for idx,player in bought.iterrows():
        if player["Owner"]==ME:
            probability=probability_of_being_in_first_team(player["OriginalName"],parsed_html)
            #print("{} {}".format(player["OriginalName"],probability) )
            all[player["Name"]]=probability
    return all
probabilities=first_team(bought)
print(probabilities)

{'CRISCITO': 90.0, 'STRAKOSHA': 90.0, 'ZIELINSKI': 55.0, 'KURTIC': 60.0, 'ZAPATAD': 80.0, 'IZZO': 90.0, 'NKOULOU': 0, 'KOULIBALY': 0, 'POLITANO': 55.0, 'KESSIE': 0, 'BONIFAZI': 60.0, 'MURRU': 85.0, 'RADU': 85.0, 'QUAGLIARELLA': 90.0, 'VERETOUT': 55.0, 'GOMEZA': 60.0, 'BONUCCI': 90.0, 'BERISHAE': 90.0, 'FLORENZI': 85.0, 'MARTINEZL': 65.0, 'CHIELLINI': 0, 'MILENKOVIC': 85.0, 'SUSO': 65.0, 'MANDRAGORA': 55.0, 'NANDEZ': 85.0, 'IAGOFALQUE': 45.0, 'BALOTELLI': 0, 'PROTO': 5.0}


In [4]:
download_html()

Server Offline


In [5]:
from amplpy import AMPL,Environment
ampl = AMPL(Environment('/home/asino/Downloads/ampl_linux-intel64/'))
ampl.setOption("solver","/home/asino/Downloads/ampl_linux-intel64/cplex")
ampl.setOption("absmipgap",1)



In [6]:
def fillDictOfBrokenPlayers():
    bought=pd.read_csv("availability-of-players.csv")
    dictOfMyPlayers={}
    dictOfBrokenPlayers={}
    for idx,player in bought.iterrows():
        if player["Owner"]==ME:
            if player["CanPlay"]==0:
                dictOfBrokenPlayers[player["Name"]]=1
            else:
                dictOfBrokenPlayers[player["Name"]]=0
            if probabilities[player["Name"]]<THRESHOLD:
                dictOfBrokenPlayers[player["Name"]]=1
    return dictOfBrokenPlayers

dictOfBrokenPlayers=fillDictOfBrokenPlayers()
print(dictOfBrokenPlayers)   

{'CRISCITO': 0, 'STRAKOSHA': 0, 'ZIELINSKI': 0, 'KURTIC': 0, 'ZAPATAD': 0, 'IZZO': 0, 'NKOULOU': 1, 'KOULIBALY': 1, 'POLITANO': 0, 'KESSIE': 1, 'BONIFAZI': 0, 'MURRU': 0, 'RADU': 0, 'QUAGLIARELLA': 0, 'VERETOUT': 0, 'GOMEZA': 0, 'BONUCCI': 0, 'BERISHAE': 0, 'FLORENZI': 0, 'MARTINEZL': 0, 'CHIELLINI': 1, 'MILENKOVIC': 0, 'SUSO': 0, 'MANDRAGORA': 0, 'NANDEZ': 0, 'IAGOFALQUE': 0, 'BALOTELLI': 1, 'PROTO': 1}


In [7]:
ampl.read("07-prepare-selection.mod")
ampl.readData("operations-mantra-online.dat")
setOfPlayers=ampl.getSet("Players")
#setOfPlayers.setValues(["CIAO"])

	07-prepare-selection.mod
	line 33 offset 991
	Caution: 0-dimensional slice


In [8]:
def analyseSolution():
    putPlayerInSlot=ampl.getVariable("putPlayerInSlot")
    cost=ampl.getParameter("cost")
    pfs=ampl.getParameter("performance")
    moduleChosen=ampl.getVariable("bidderChoosesModule")
    globalPerf=ampl.getVariable("minPerf").value()
    
    roles=ampl.getSet("CompatibilityPlayerRole").getValues().toDict()
    slots=ampl.getSet("CompatibilitySlotRole").getValues().toDict()
    solution=putPlayerInSlot.getValues().toDict()
    costs=cost.getValues().toDict()
    pfs=pfs.getValues().toDict()
    modules=moduleChosen.getValues().toDict()
    moduleUsed="AAA"
    

    for moduleName,taken in modules.items():
        if taken>0.5:
            #print("Module: {}\n\n".format(moduleName))
            moduleUsed=moduleName
    myPls={}
    for key,val in solution.items():

        if key[0]==moduleUsed:
            if val>0.5:
                myPls[key[2]]=key[1]
                #print(key,val)
    rolesA={}

    #To show the roles of each player
    for key,val in roles.items():
        if key[0] not in rolesA.keys():
            rolesA[key[0]]=[]
        rolesA[key[0]].append(key[1])


    mappingSlotNumberToPossibleRoles={}
    for key,val in slots.items():
        if key[0]==moduleUsed:

            slotNr=key[1]
            role=key[2]
            if slotNr not in mappingSlotNumberToPossibleRoles.keys():
                mappingSlotNumberToPossibleRoles[slotNr]=[]
            mappingSlotNumberToPossibleRoles[slotNr].append(role)

    emptySlotsInField=0
    emptySlotsOutside=0
    message=""
    texts=[]
    for slotNr,possibleRoles in mappingSlotNumberToPossibleRoles.items():
        if slotNr<=11:
            privilege="INS"
        else:
            privilege="OUT"
        try:
            message+=("{} {} plays  in slot {}  \n".format(privilege, myPls[slotNr], possibleRoles))
            texts.append("{} {} plays  in slot {}\n".format(privilege,possibleRoles,myPls[slotNr]))
        except KeyError:
            if privilege=="TOP":
                emptySlotsInField+=1
            else:
                emptySlotsOutside+=1
    texts=sorted(texts)
    message="".join(texts)
    return globalPerf,moduleUsed,message,emptySlotsInField,emptySlotsOutside
_,_,msg,_,_=analyseSolution()


In [9]:
def suggestTeam(dictOfBrokenPlayers):
    plas=ampl.getSet("Players").getValues()
    valuesForBrokenPlayersAmpl={}
    for name in plas:
        name=name[0]
        #print(name)
        valuesForBrokenPlayersAmpl[name]=0
    for player,value in dictOfBrokenPlayers.items():
        valuesForBrokenPlayersAmpl[player]=value
    ampl.getParameter("broken").setValues(valuesForBrokenPlayersAmpl)
        
        
        
    
    ampl.solve()
suggestTeam(dictOfBrokenPlayers)
_,mod,msg,inf,_=analyseSolution()
print(inf)
print(mod)
print(msg)

CPLEX 12.8.0.0: optimal integer solution; objective 84.37230554
1684 MIP simplex iterations
0 branch-and-bound nodes
0
m4231
INS ['A', 'Pc'] plays  in slot ZAPATAD
INS ['Dc'] plays  in slot CRISCITO
INS ['Dc'] plays  in slot IZZO
INS ['Dd'] plays  in slot MILENKOVIC
INS ['Ds'] plays  in slot MURRU
INS ['M', 'C'] plays  in slot ZIELINSKI
INS ['M'] plays  in slot MANDRAGORA
INS ['Por'] plays  in slot STRAKOSHA
INS ['T'] plays  in slot GOMEZA
INS ['W', 'A'] plays  in slot SUSO
INS ['W'] plays  in slot POLITANO
OUT ['A', 'Pc'] plays  in slot QUAGLIARELLA
OUT ['Dc'] plays  in slot BONIFAZI
OUT ['Dc'] plays  in slot BONUCCI
OUT ['Dd'] plays  in slot FLORENZI
OUT ['Ds'] plays  in slot RADU
OUT ['M', 'C'] plays  in slot VERETOUT
OUT ['M'] plays  in slot NANDEZ
OUT ['Por'] plays  in slot BERISHAE
OUT ['T'] plays  in slot KURTIC
OUT ['W', 'A'] plays  in slot IAGOFALQUE



In [10]:
'''
pls=dictOfBrokenPlayers.keys()
dictOfEffects={}
solidPlayers=["STRAKOSHA","BERISHAE","IZZO"]
for player in pls:
    for p2 in pls:
        if player >p2:
            if player not in solidPlayers and p2 not in solidPlayers:
                broks=fillDictOfBrokenPlayers()
                broks[player]=1
                broks[p2]=1
                suggestTeam(broks)
                dictOfEffects[(player,p2)]=(analyseSolution())
#suggestTeam({},{})
'''

'\npls=dictOfBrokenPlayers.keys()\ndictOfEffects={}\nsolidPlayers=["STRAKOSHA","BERISHAE","IZZO"]\nfor player in pls:\n    for p2 in pls:\n        if player >p2:\n            if player not in solidPlayers and p2 not in solidPlayers:\n                broks=fillDictOfBrokenPlayers()\n                broks[player]=1\n                broks[p2]=1\n                suggestTeam(broks)\n                dictOfEffects[(player,p2)]=(analyseSolution())\n#suggestTeam({},{})\n'

In [11]:
'''
couples=dictOfEffects.keys()
cp=sorted(couples,key=lambda k:dictOfEffects[k][0])
for i in range(10):
    print(cp[i],dictOfEffects[cp[i]][0])
    #print(dictOfEffects[cp[i]][2])
    #break
'''

'\ncouples=dictOfEffects.keys()\ncp=sorted(couples,key=lambda k:dictOfEffects[k][0])\nfor i in range(10):\n    print(cp[i],dictOfEffects[cp[i]][0])\n    #print(dictOfEffects[cp[i]][2])\n    #break\n'